In [ ]:
import findspark
findspark.find()
findspark.init()

In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
    .master("local") \
    .appName("StockMarket") \
    .config("spark.executor.memory", "6gb") \
    .getOrCreate()

In [ ]:
df =spark.read.format('com.databricks.spark.csv')\
                    .options(header='true', inferschema='true')\
                    .load('AAPL.csv')

In [ ]:
df.show()

In [ ]:
import pyspark.sql.functions as f
df = df.withColumn('date', f.to_date('Date'))

In [ ]:
df.show(n=5)

In [ ]:
date_breakdown = ['year', 'month', 'day']
for i in enumerate(date_breakdown):
    index = i[0]
    name = i[1]
    df = df.withColumn(name, f.split('date', '-')[index])

df.show(n=10)

In [ ]:
df_plot = df.select('year', 'Adj Close').toPandas()

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
df_plot.set_index('year', inplace=True)
df_plot.plot(figsize=(16, 6), grid=True)
plt.title('Apple stock')
plt.ylabel('Stock Quote ($)')
plt.show()

In [ ]:
df.toPandas().shape

In [ ]:
df.dropna().count()

In [ ]:
df.select('Open', 'High', 'Low', 'Close', 'Adj Close').describe().show()

In [ ]:
df.groupBy(['year']).agg({'Adj Close':'count'})\
    .withColumnRenamed('count(Adj Close)', 'Row Count')\
    .orderBy(["year"],ascending=False)\
    .show()

In [ ]:
trainDF = df[df.year < 2017]
testDF = df[df.year > 2016]

In [ ]:
trainDF.toPandas().shape

In [ ]:
testDF.toPandas().shape

In [ ]:
trainDF_plot = trainDF.select('year', 'Adj Close').toPandas()
trainDF_plot.set_index('year', inplace=True)
trainDF_plot.plot(figsize=(16, 6), grid=True)
plt.title('Apple Stock 2000-2016')
plt.ylabel('Stock Quote ($)')
plt.show()

In [ ]:
testDF_plot = testDF.select('year', 'Adj Close').toPandas()
testDF_plot.set_index('year', inplace=True)
testDF_plot.plot(figsize=(16, 6), grid=True)
plt.title('Apple Stock 2017-2018')
plt.ylabel('Stock Quote ($)')
plt.show()

In [ ]:
import numpy as np
trainArray=np.array(trainDF.select('Open','High','Low','Close','Volume','Adj Close').collect())
testArray=np.array(testDF.select('Open','High','Low','Close','Volume','Adj Close').collect())

In [ ]:
print(trainArray[0])
print('---------------')
print(testArray[0])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minMaxScale = MinMaxScaler()

In [ ]:
minMaxScale.fit(trainArray)

In [ ]:
testingArray = minMaxScale.transform(testArray)
trainingArray = minMaxScale.transform(trainArray)

In [ ]:
print(testingArray[0])
print('---------------')
print(trainingArray[0])

In [ ]:
xtrain = trainingArray[:, 0:-1]
xtest = testingArray[:, 0:-1]
ytrain = trainingArray[:, -1:]
ytest = testingArray[:, -1:]

In [ ]:
trainingArray[0]

In [ ]:
xtrain[0]

In [ ]:
ytrain[0]

In [ ]:
print('xtrain shape = {}'.format(xtrain.shape))
print('xtest shape = {}'.format(xtest.shape))
print('ytrain shape = {}'.format(ytrain.shape))
print('ytest shape = {}'.format(ytest.shape))

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(xtrain[:,0],color='red', label='open')
plt.plot(xtrain[:,1],color='blue', label='high')
plt.plot(xtrain[:,2],color='green', label='low')
plt.plot(xtrain[:,3],color='purple', label='close')
plt.legend(loc = 'upper left')
plt.title('Open, High, Low, and Close by Day')
plt.xlabel('Days')
plt.ylabel('Scaled Quotes')
plt.show()

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(xtrain[:,4],color='black', label='volume')
plt.legend(loc = 'upper right')
plt.title('Volume by Day')
plt.xlabel('Days')
plt.ylabel('Scaled Volume')
plt.show()

In [ ]:
from keras import models, layers

In [ ]:
model = models.Sequential()
model.add(layers.LSTM(1, input_shape=(1,5)))
model.add(layers.Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
xtrain = xtrain.reshape((xtrain.shape[0], 1, xtrain.shape[1]))
xtest = xtest.reshape((xtest.shape[0], 1, xtest.shape[1]))

In [ ]:
print('The shape of xtrain is {}: '.format(xtrain.shape))
print('The shape of xtest is {}: '.format(xtest.shape))

In [ ]:
loss = model.fit(xtrain, ytrain, batch_size=10, epochs=10)

In [ ]:
plt.plot(loss.history['loss'], label = 'loss')
plt.title('mean squared error by epoch')
plt.legend()
plt.show()

In [ ]:
predicted = model.predict(xtest)

In [ ]:
combined_array = np.concatenate((ytest, predicted), axis = 1)

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(combined_array[:,0],color='red', label='actual')
plt.plot(combined_array[:,1],color='blue', label='predicted')
plt.legend(loc = 'lower right')
plt.title('2017 Actual vs. Predicted APPL Stock')
plt.xlabel('Days')
plt.ylabel('Scaled Quotes')
plt.show()

In [ ]:
import sklearn.metrics as metrics
np.sqrt(metrics.mean_squared_error(ytest,predicted))

In [ ]:
spark.stop()